# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [2]:
# Your solution goes here

import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_books(min_rating, max_price):
    base_url = "http://books.toscrape.com/catalogue/page-{}.html"
    
    def get_star_rating(star_class):
        if "One" in star_class:
            return 1
        elif "Two" in star_class:
            return 2
        elif "Three" in star_class:
            return 3
        elif "Four" in star_class:
            return 4
        elif "Five" in star_class:
            return 5
        return 0

    books_data = []
    page = 1
    
    while True:
        response = requests.get(base_url.format(page))
        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.content, 'html.parser')
        books = soup.find_all('article', class_='product_pod')

        if not books:
            break

        for book in books:
            title = book.h3.a['title']
            price = float(book.find('p', class_='price_color').text[1:])
            rating = get_star_rating(book.p['class'])
            availability = book.find('p', class_='instock availability').text.strip()
            
            # Only process books matching the filters
            if rating >= min_rating and price <= max_price:
                # Get book details page
                book_url = "http://books.toscrape.com/catalogue/" + book.h3.a['href']
                book_response = requests.get(book_url)
                book_soup = BeautifulSoup(book_response.content, 'html.parser')

                # Extract book details
                upc = book_soup.find('th', text='UPC').find_next_sibling('td').text
                genre = book_soup.find('ul', class_='breadcrumb').find_all('a')[-1].text
                description = book_soup.find('meta', {'name': 'description'})
                description = description['content'].strip() if description else "No description available"

                # Add book data to the list
                books_data.append({
                    'UPC': upc,
                    'Title': title,
                    'Price (£)': price,
                    'Rating': rating,
                    'Genre': genre,
                    'Availability': availability,
                    'Description': description
                })

        page += 1

    # Convert to DataFrame
    return pd.DataFrame(books_data)

# Example usage
if __name__ == "__main__":
    df = scrape_books(min_rating=4, max_price=20)
    print(df)
    # Save to CSV for analysis
    df.to_csv("books_to_scrape_results.csv", index=False)


C:\Users\pboat\AppData\Local\Temp\ipykernel_29940\1162425012.py:51: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = book_soup.find('th', text='UPC').find_next_sibling('td').text


                 UPC                                              Title  \
0   ce6396b0f23f6ecc                                        Set Me Free   
1   6258a1f6a6dcfe50  The Four Agreements: A Practical Guide to Pers...   
2   6be3beb0793a53e7                                     Sophie's World   
3   657fe5ead67a7767            Untitled Collection: Sabbath Poems 2014   
4   51653ef291ab7ddc                                    This One Summer   
..               ...                                                ...   
70  9c96cd1329fbd82d                                    The Zombie Room   
71  b78deb463531d078                                    The Silent Wife   
72  4280ac3eab57aa5d                                  The Girl You Lost   
73  29fc016c459aeb14              The Edge of Reason (Bridget Jones #2)   
74  19fec36a1dfb4c16  A Spy's Devotion (The Regency Spies of London #1)   

    Price (£)  Rating               Genre Availability  \
0       17.46       5         Young Adult

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape_books(min_rating, max_price):
    base_url = "http://books.toscrape.com/catalogue/page-{}.html"

    # Convert star rating text to numerical values
    rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

    books_data = []
    page = 1

    while True:
        try:
            response = requests.get(base_url.format(page), timeout=10)
            response.raise_for_status()  # Raises an error for bad status codes
        except requests.RequestException as e:
            print(f"Error fetching page {page}: {e}")
            break

        soup = BeautifulSoup(response.content, 'html.parser')
        books = soup.find_all('article', class_='product_pod')

        if not books:
            break

        for book in books:
            title = book.h3.a['title']
            price_text = book.find('p', class_='price_color').text
            price = float(re.search(r"[\d.]+", price_text).group())  # Extract numeric price
            rating_class = book.p.get("class", [])
            rating = rating_map.get(rating_class[-1], 0)  # Extract rating

            availability = book.find('p', class_='instock availability').text.strip()

            # Only process books matching the filters
            if rating >= min_rating and price <= max_price:
                book_url = "http://books.toscrape.com/catalogue/" + book.h3.a['href']
                
                try:
                    book_response = requests.get(book_url, timeout=10)
                    book_response.raise_for_status()
                    book_soup = BeautifulSoup(book_response.content, 'html.parser')

                    # Extract book details
                    upc = book_soup.find('th', text='UPC').find_next_sibling('td').text
                    genre = book_soup.find('ul', class_='breadcrumb').find_all('a')[-1].text
                    description_tag = book_soup.find('meta', {'name': 'description'})
                    description = description_tag['content'].strip() if description_tag else "No description available"

                    # Store book data
                    books_data.append({
                        'UPC': upc,
                        'Title': title,
                        'Price (£)': price,
                        'Rating': rating,
                        'Genre': genre,
                        'Availability': availability,
                        'Description': description
                    })

                except requests.RequestException as e:
                    print(f"Error fetching book details {book_url}: {e}")

        page += 1  # Move to the next page

    # Convert to DataFrame
    return pd.DataFrame(books_data)

# Example usage
if __name__ == "__main__":
    df = scrape_books(min_rating=4, max_price=20)
    print(df.head())  # Display first few rows
    df.to_csv("books_to_scrape_results.csv", index=False)  # Save results


C:\Users\pboat\AppData\Local\Temp\ipykernel_29940\880036900.py:48: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = book_soup.find('th', text='UPC').find_next_sibling('td').text


Error fetching page 51: 404 Client Error: Not Found for url: http://books.toscrape.com/catalogue/page-51.html
                UPC                                              Title  \
0  ce6396b0f23f6ecc                                        Set Me Free   
1  6258a1f6a6dcfe50  The Four Agreements: A Practical Guide to Pers...   
2  6be3beb0793a53e7                                     Sophie's World   
3  657fe5ead67a7767            Untitled Collection: Sabbath Poems 2014   
4  51653ef291ab7ddc                                    This One Summer   

   Price (£)  Rating           Genre Availability  \
0      17.46       5     Young Adult     In stock   
1      17.66       5    Spirituality     In stock   
2      15.94       5      Philosophy     In stock   
3      14.27       4          Poetry     In stock   
4      19.49       4  Sequential Art     In stock   

                                         Description  
0  Aaron Ledbetter’s future had been planned out ...  
1  In The Four A